# SEL634 - Laboratório de Robôs Manipuladores

## Inicialização da biblioteca

Carregar a biblioteca de robótica para Python:

In [ ]:
!python3 -m pip install --user --upgrade pip==20.1
!pip install roboticstoolbox-python
!pip install spatialmath-python

Requirement already up-to-date: pip==20.1 in /root/.local/lib/python3.7/site-packages (20.1)
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [ ]:
import numpy as np
#import roboticstoolbox as rtb
from roboticstoolbox import *
from spatialmath import *
from math import pi
import matplotlib.pyplot as plt
from matplotlib import cm
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

# no Binder utilizar essa linha
%matplotlib notebook  
# no Colab utilizar essa linha
# %matplotlib inline

## Cinemática Direta do Robô Kinova Gen3

O robô Kinova Gen3 pode ser modelado pela seguinte função (ver Tarefa 2):

In [ ]:
def kinova_robot(my_name):
  robot = DHRobot(
      [
       # Colocar seu código aqui para completar a descrição do robô
       ], name=my_name)
  return robot


A pose do efetuador do robô Kinova Gen3 é dada em relação a base, $B$, pela função:

In [ ]:
def pose_efetuador(robot, q):
  H_B0 = SE3.Rz(0)*SE3(0,0,0)*SE3(0,0,0)*SE3.Rx(pi)
  H_07 = robot.fkine(q)
  return H_B0*H_07

## Trajetória do Robô no Espaço das Juntas

Considere que a junta $i$ possui a seguinte trajetória  em graus no tempo que leva a junta $i$ da posição $\theta_{si}$ até $\theta_{gi}$ em um dado tempo $T_d$:
$$ \theta_i (t) = c_{0i} + c_{1i} t + c_{2i} t^2 + c_{3i} t^3 + c_{4i} t^4 + c_{5i} t^5 $$
Para velocidades e acelerações iniciais e finais iguais a zero, os coeficientes do polinômio são dados em função da posição inicial e final e do intervalo de tempo:
\begin{eqnarray*}
	&c_{0i}& = \theta_{si}\\
	&c_{1i}& = c_{2i} = 0\\
	&c_{3i}& = \frac{10(\theta_{gi}-\theta_{si})}{T_d^{3}}\\
	&c_{4i}& = \frac{15(\theta_{si}-\theta_{gi})}{T_d^{4}}\\
	&c_{5i}& = \frac{6(\theta_{gi}-\theta_{si})}{T_d^{5}}
\end{eqnarray*}

A velocidade de cada junta para condições iniciais nulas é dada por:
$$ \dot\theta_i (t) = c_{1i} + 2c_{2i} t + 3c_{3i} t^2 + 4c_{4i} t^3 + 5c_{5i} t^4 $$

A aceleração de cada junta para condições iniciais nulas é dada por:
$$ \ddot\theta_i (t) = 2c_{2i} + 6c_{3i} t + 12c_{4i} t^2 + 20c_{5i} t^3 $$

A função a seguir foi implementada na Tarefa 3 para calcular a trajetória das juntas do manipulador:


In [ ]:
def calcular_trajetoria(theta_s, theta_g, Td, dt):
  # Coloque seu codigo aqui

  return theta_t, dtheta_t, ddtheta_t

A função a seguir foi implementada na Tarefa 3 para calcular a sequencia no tempo de matrizes de transformação homogênea que representa a pose do robô (posição e orientação do efetuador)

In [ ]:
def calcular_traj_efetuador(robot, theta_t):
  # Coloque seu codigo aqui

return H_efetuador

## Velocidade do Efetuador do Robô Manipulador

Na tarefa 4 você implementou as seguintes funções para calcular o Jacobiano do manipulador: 


In [ ]:
def calcular_jacobiano(robot,theta):
  # Coloque seu codigo aqui

  return J_efetuador

Na tarefa 4, você implementou uma função que calcula a velocidade do efetuador do manipulador Kinova Gen3 em relação a sua base $B$.

O calculo das velocidades linear e angular do manipulador em relação ao sistema $\{0\}$ para um dado instante de tempo é:

$$\begin{bmatrix} ^0v_n(t) \\ ^0w_n(t) \end{bmatrix} = {^0J_n}(q(t)) ~\dot q(t) $$

Como a origem do sistema de coordenadas $\{0\}$ e $\{B\}$ coincidem, para transformar o vetor de velocidades para o sistema de coordenadas da base $\{B\}$:

$$ \begin{bmatrix} ^Bv_n(t) \\ ^Bw_n(t) \end{bmatrix} = {^BR_0}\begin{bmatrix} ^Bv_n(t) \\ ^Bw_n(t) \end{bmatrix} $$



In [ ]:
def calcular_vel_efetuador(robot,theta_t,dtheta_t):
  # Coloque seu codigo aqui


  return V_efetuador

## 5. Controle de Trajetória do Efetuador no Espaço Cartesiano

**Aviso:** Esta é uma versão preliminar do documento que está sujeita a revisão. Estou procurando deixar a descrição desta tarefa mais clara e completa.

Nesta tarefa você irá implementar um conjunto de funções que possibilitam o controle da posição do efetuador no espaço cartesiano. Isso permite que o efetuador possa descrever uma trajetória reta no espaço 3D cartesiano.

### 5.1 Descrição de uma trajetória reta no espaço cartesiano

Considere uma trajetória descrita por um seguimento de reta no espaço cartesiano que tem início no ponto 3D, $\mathbf{^Bx_s} = [x_s, y_s, z_s]^T$, e término no ponto 3D, $\mathbf{^Bx_g} = [x_g, y_g, z_g]^T$, ambos representados no sistema de coordenadas da base $\{B\}$. A velocidade linear do efetuador no inicio e fim da trajetória devem ser iguais a $0$. 

Quando a velocidade inicial e final do ponto é zero, a trajetória do ponto 3D a ser executada pode ser representada por um polinômio de quinta ordem no tempo:

$$ \mathbf{x} (t) = c_{0} + c_{1} t + c_{2} t^2 + c_{3} t^3 + c_{4} t^4 + c_{5} t^5 $$
Para velocidades e acelerações iniciais e finais iguais a zero, os coeficientes do polinômio são dados em função da posição inicial e final e do intervalo de tempo $T_d$:
\begin{eqnarray*}
	&c_{0}& = \mathbf{x_i}\\
	&c_{1}& = c_{2i} = 0\\
	&c_{3}& = \frac{10(\mathbf{x_g}-\mathbf{x_s})}{T_d^{3}}\\
	&c_{4}& = \frac{15(\mathbf{x_s}-\mathbf{x_g})}{T_d^{4}}\\
	&c_{5}& = \frac{6(\mathbf{x_g}-\mathbf{x_s})}{T_d^{5}}
\end{eqnarray*}

Observe que $\mathbf{x_s}$ e $\mathbf{x_g}$ tem dimensão $3 \times 1$, e portanto, os coeficientes $c_0$, $c_1$, $c_2$, $c_3$, $c_4$, $c_5$ também possuem dimensão $3 \times 1$. 

A velocidade linear do efetuador para condições iniciais nulas é dada por:
$$ \mathbf{\dot  x} (t) = c_{1} + 2c_{2} t + 3c_{3} t^2 + 4c_{4} t^3 + 5c_{5} t^4 $$

Importante ressaltar, que a trajetória de quinta ordem no espaço cartesianos não será uma reta quando as velocidades iniciais ou finais foram diferentes de zero.

Para que não seja necessário utilizar a cinemática inversa do robô manipulador, nessa tarefa você irá considerar que a configuração inicial e final do robô foram dadas, ou seja, $\mathbf{q_s}$ e $\mathbf{q_g}$. Então, utilizando a função `pose_efetuador(robot, q)`, você irá obter a posição inicial $\mathbf{x_s}$ e posição final $\mathbf{x_g}$ do efetuador.

Complete o código a seguir para implementar uma função que retorna os pontos $\mathbf{x}(t)$ da trajetória do efetuador e as velocidades lineares do efetuador $\mathbf{\dot x}(t)$.




In [ ]:
def calcular_trajetoria_cartesiano(robot, q_s, q_g, Td, dt):
  # Coloque seu codigo aqui


  return x_t, dx_t

Complete o código a seguir para implementar uma função que plota a trajetória no espaço cartesiano.

In [ ]:
def plotar_trajetoria_cartesiano(x_t)



### 5.2 Conversão de trajetória no espaço cartesiano para espaço das juntas

**Método 1:** uma das formas de converter uma trajetória no espaço cartesiano para o espaço das juntas é aplicar a cinemática inversa para todos os pontos da trajetória $\mathbf{x}(t)$. Entretanto, para o manipulador redundante, para uma dada posição do efetuador, existem múltiplas configurações possíveis (soluções válidas da cinemática inversa). Portanto, é preciso aplicar um método numérico iterativo que retorna, para um ponto da trajetória, a solução de configuração *mais próxima* da configuração associada ao ponto anterior da trajetória. Dessa forma, a continuidade do movimento das juntas é preservada.

**Método 2:** Uma outra forma de fazer isso é utilizar a pseudo-inversa do Jacobiano. Entretanto, esse método não é adequado quando a trajetória passa próximo de um ponto de singularidade. Próximo da singulariade, o valor da pseudo-inversa do Jacobinado é muito alto, o que resultaria em uma velocidade de juntas também muito alta. 

Nessa tarefa, você irá aplicar o **método 2** para converter a trajetória no espaço cartesiano para o espaço das juntas.

Se o determinante do Jacobiano de velocidade linear $|J_v|$ for maior que um limiar, ou seja, $|J_v| > J_{min}$, então o ponto está suficientemente longe da singularidade, e podemos calcular a pseudo-inversa $J_v^{-1}$. 

Então, para construir a trajetória no espaço das juntas:

$$ \mathbf{q}(0) = \mathbf{q_s}$$

Para $t = 0 \ldots (T_d-dt)$:

$$c_J(t) = |J_v(\mathbf{q}(t))|$$
$$\mathbf{\dot q}(t) = J_v^{-1}(\mathbf{q}(t)) \mathbf{\dot x}(t) $$
$$\delta \mathbf{q} = J_v^{-1}(\mathbf{q}(t)) [\mathbf{x}(t+dt) - \mathbf{x}(t)]$$
$$\mathbf{q}(t+dt) = \mathbf{q}(t) + \delta \mathbf{q}$$

Implementar a seguinte função de conversão que retorna uma matriz com as configurações do robô para cada instante de tempo. Não se preocupe, se a configuração final da trajetória não for a mesma que $q_g$. O importante é que a posição final do efetuador seja igual a posição final desejada.

In [ ]:
def trajetoria_cartesiano_para_juntas(robot,q_s, q_g, x_t, dx_t, Td, dt, J_min)



return q_t, dq_t, cJ_t

### 5.3 Teste as funções implementadas

Escolha uma configuração inicial $\mathbf{q_s}$ e final para o robô $\mathbf{q_g}$ de forma que não exista pontos de singularidade ao longo da trajetória reta entre as configuracões. Escolha também um valor de $T_d$ e $dt$.

Teste as funções implementadas em 5.1 e 5.2 para os valores escolhidos, e plote um gráfico com a trajetória 3D do efetuador no espaço cartesiano, e um gráfico com a trajetória no espaço das juntas do manipulador. 